In [1]:
%pip install openai
%pip install PyMuPDF 
%pip install pillow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from openai import OpenAI
import dotenv
import os

github_token = os.environ["GITHUB_KEY"]
if not github_token:
    raise ValueError("GITHUB_TOKEN is not set in your system environment variables")

# Set the required OpenAI environment variables using the retrieved token
os.environ["OPENAI_API_KEY"] = github_token
os.environ["OPENAI_BASE_URL"] = "https://models.inference.ai.azure.com/"

print("Environment variables have been set successfully.")

GPT_MODEL = "gpt-4o-mini"

client = OpenAI()

Environment variables have been set successfully.


In [ ]:
from openai import OpenAI
import fitz  # PyMuPDF
import io
import os
from PIL import Image
import base64
import json

api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)


@staticmethod
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def pdf_to_base64_images(pdf_path):
    #Handles PDFs with multiple pages
    pdf_document = fitz.open(pdf_path)
    base64_images = []
    temp_image_paths = []

    total_pages = len(pdf_document)

    for page_num in range(total_pages):
        page = pdf_document.load_page(page_num)
        pix = page.get_pixmap()
        img = Image.open(io.BytesIO(pix.tobytes()))
        temp_image_path = f"temp_page_{page_num}.png"
        img.save(temp_image_path, format="PNG")
        temp_image_paths.append(temp_image_path)
        base64_image = encode_image(temp_image_path)
        base64_images.append(base64_image)

    for temp_image_path in temp_image_paths:
        os.remove(temp_image_path)

    return base64_images

In [16]:
def summarize_page_data(base64_image):
    system_prompt = f"""
    You are an OCR-like data extraction tool that extracts data from textbook PDFs.
   
    1. Please extract the data from this textbook page and output 5 sentences that describe the main points of this page. Do not number or bullet the sentences.

    2. The text should only contain the sentences and no title.

    3. If the page contains no relevant text data, please output an empty text object and don't make up any data.

    4. If there are blank data fields in the invoice, please include them as "null" values in the text object.

    5. Ignore tables and only pay attention to captions. If a page is only a table try to create a sentence summarizing the table.
            
    6. Don't interpolate or make up data.

    7. If there is not enough text data to form 5 sentences that are distinct in meaning, you can choose to output less sentences.

    """
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        response_format={ "type": "text" },
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "extract the data in this textbook chapter and output into text "},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image}", "detail": "high"}}
                ]
            }
        ],
        temperature=0.0,
    )
    return response.choices[0].message.content


In [ ]:
""" PDF TO 5 SENTENCES """

def extract_from_multiple_pages(base64_images, original_filename, output_directory):
    entire_invoice = []

    for base64_image in base64_images:
        # Extract data for each image
        invoice_text = summarize_page_data(base64_image)
        
        # Since it's plain text, append it to the list
        entire_invoice.append(invoice_text.strip())

    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)

    # Construct the output file path
    output_filename = os.path.join(output_directory, original_filename.replace('.pdf', '_extracted.txt'))
    
    # Save the entire_invoice list as a plain text file
    with open(output_filename, 'w', encoding='utf-8') as f:
        for page_text in entire_invoice:
            f.write(page_text + '\n\n')  # Separate pages with a double newline
    
    return output_filename

def main_extract(read_path, write_path):
    # to save tokens we will only do the first 3 invoices
    for filename in os.listdir(read_path)[:3]:
        file_path = os.path.join(read_path, filename)
        print(f"Extracting data from {file_path}")
        if os.path.isfile(file_path):
            base64_images = pdf_to_base64_images(file_path)
            extract_from_multiple_pages(base64_images, filename, write_path)

read_path= "./data/textdata"
write_path= "./data/output"

main_extract(read_path, write_path)

In [24]:
import os
import openai

# Function to summarize text using OpenAI's GPT API

def summarize_text_file(input_file_path, output_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        text_data = file.read().strip()

    system_prompt = f"""
    You are an OCR-like data extraction tool that extracts data from textbook PDFs.
   
    1. Please summarize the content and output 10 sentences that describe the main points. Do not number or bullet the sentences.

    2. The text should only contain the sentences and no title.

    3. If the text contains no relevant information, please output an empty text object and don't make up any data.

    4. Ignore tables and focus only on meaningful captions or descriptive content.

    5. Don't interpolate or make up data.

    6. If there is not enough text data to form 10 sentences that are distinct in meaning, you can choose to output fewer sentences.

    """
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        response_format={ "type": "text" },
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "extract the data in this textbook chapter and output into text "},
                    {"type": "text", "text": f"{text_data}"}
                ]
            }
        ],
        temperature=0.0,
    )
    text = response.choices[0].message.content

    text = text.strip()

    with open(output_file_path, 'w', encoding='utf-8') as f:
        f.write(text)


# Main function to summarize all text files in a directory
def summarize_all_files(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for filename in os.listdir(input_dir):
        if filename.endswith('.txt'):
            input_file_path = os.path.join(input_dir, filename)
            output_file_path = os.path.join(output_dir, filename.replace('.txt', '_summary.txt'))
            summarize_text_file(input_file_path, output_file_path)

# Paths for input and output directories
input_directory = "./data/output"
output_directory = "./data/output_summary"

# Run the script
summarize_all_files(input_directory, output_directory)
